In [1]:
!pip install keras_tuner -q

# import cryptocompare as cc
import requests
import IPython
import yaml
import json
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

# alternatively if you have not stored your API key in a separate file
api_key = 'b234d4305f5d1c5b9bb158e2cbb35da6703d86dd76e6d5c2f4b2fe4655ab3104'

# Number of past days for which we retrieve data
data_limit = 2000

# Define coin symbols
symbol_a = 'BTC'
symbol_b = 'ETH'

# Query price data

# Generic function for an API call to a given URL
def api_call(url):
  # Set API Key as Header
  headers = {'authorization': 'Apikey ' + api_key,}
  session = requests.Session()
  session.headers.update(headers)

  # API call to cryptocompare
  response = session.get(url)

  # Conversion of the response to dataframe
  historic_blockdata_dict = json.loads(response.text)
  df = pd.DataFrame.from_dict(historic_blockdata_dict.get('Data').get('Data'), orient='columns', dtype=None, columns=None)
  return df

def prepare_pricedata(df):
  df['date'] = pd.to_datetime(df['time'], unit='s')
  df.drop(columns=['time', 'conversionType', 'conversionSymbol'], inplace=True)
  return df

# Load the price data
base_url = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym='


C=prepare_pricedata(api_call(f'{base_url}{symbol_a}&tsym=USD&limit={data_limit}&toTs={1678838400}'))
B=prepare_pricedata(api_call(f'{base_url}{symbol_a}&tsym=USD&limit={data_limit}&toTs={1506038400}'))
A=prepare_pricedata(api_call(f'{base_url}{symbol_a}&tsym=USD&limit={data_limit}&toTs={1333238400}'))

price_data = pd.concat([A[:-1],B[:-1],C]).reset_index(drop = True)
df = pd.DataFrame(price_data.close)
# get the index of the first non-null row
first_nonzero_row = df.index[df.apply(lambda x: x.ne(0)).any(1)][0]

price_data = price_data.set_index('date')[first_nonzero_row:]

# Prepare the onchain dataframe
def prepare_onchain_data(df):
  # replace the timestamp with a data and filter some faulty values
  df['date'] = pd.to_datetime(df['time'], unit='s')
  df.drop(columns='time', inplace=True)
  df = df[df['hashrate'] > 0.0]
  return df

# Load onchain data for Bitcoin
base_url = 'https://min-api.cryptocompare.com/data/blockchain/histo/day?fsym='

C = prepare_onchain_data(api_call(f'{base_url}{symbol_a}&limit={data_limit}&toTs={1678838400}'))
B = prepare_onchain_data(api_call(f'{base_url}{symbol_a}&limit={data_limit}&toTs={1506038400}'))
A = prepare_onchain_data(api_call(f'{base_url}{symbol_a}&limit={data_limit}&toTs={1333238400}'))

on_chain_data = pd.concat([A[:-1],B[:-1],C]).reset_index(drop = True)
on_chain_data = on_chain_data.iloc[:,2:].set_index('date')


data = price_data.join(on_chain_data)
data.fillna(method='ffill', inplace=True)
data

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data)

# Split data
train_size = int(len(data_set_scaled) * 0.8)
train_data = data_set_scaled[0:train_size]
test_data = data_set_scaled[train_size:len(data_set_scaled)]

# Define window size
window_size = 30

# Prepare data for LSTM model
def prepare_data(data, window_size):     
    X, Y = [], []  
    
    for i in range(len(data)-window_size-1):                        
        X.append(data[i:(i+window_size),:])        
        Y.append(data[i + window_size,5])
    return np.array(X), np.array(Y)


train_X, train_Y = prepare_data(train_data, window_size)
test_X, test_Y = prepare_data(test_data, window_size)


print(train_X.shape,test_X.shape)
print(train_Y.shape,test_Y.shape)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.6 MB/s eta 0:00:00


<ipython-input-1-432031157321>:56: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  first_nonzero_row = df.index[df.apply(lambda x: x.ne(0)).any(1)][0]


(3669, 30, 20) (894, 30, 20)
(3669,) (894,)
